<a href="https://colab.research.google.com/github/JamshedAl/Email_Spam_Detection_Using_Logistic_Regression/blob/main/Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will see how to **fine-tune** one of the **hugging-face Transformers mode**l for translating **Urdu** to **English** language.

## Install required packages

In [1]:
# ! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [2]:
pip install transformers

In [3]:
pip install sentencepiece

## Importing Libraries

In [4]:
pip install accelerate -U

In [5]:
pip install datasets


In [6]:
import transformers
import datasets
from datasets import Dataset, load_metric
from google.colab import drive
print(transformers.__version__)

4.35.2


## Define Model Checkpoint

In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-ur-en"

## Setup Google Drive

In [8]:
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/fine_tuning_ur_en_machine_translation/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/fine_tuning_ur_en_machine_translation/'
/content


## Get Current Working Directory

# New Section

In [9]:
import os
my_dir = os.getcwd() + '/'
print(my_dir)

/content/


## Loading the dataset

In [10]:
# Path to translation file
path_to_data = '/content/drive/MyDrive/urd.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8')
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
ur_en_pairs = [sentence.split('\t') for sentence in  raw_data]
del(ur_en_pairs[1143])

## Creating Custom DataSet

In [11]:
training_dataset = ur_en_pairs[:1000]
validation_dataset = ur_en_pairs[1000:1100]
testing_dataset = ur_en_pairs[1100:]

In [12]:
train_dict = [{'ur': i[1], 'en': i[0]} for i in training_dataset]
test_dict = [{'ur': i[1], 'en': i[0]} for i in testing_dataset]
validate_dict = [{'ur': i[1], 'en': i[0]} for i in validation_dataset]

In [13]:
train_dict = {'translation': train_dict}
test_dict = {'translation': test_dict}
validate_dict = {'translation': validate_dict}

In [14]:
train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)
validate_dataset = Dataset.from_dict(validate_dict)


In [15]:
final_dataset = datasets.DatasetDict({"train":train_dataset, "validation":validate_dataset, "test":test_dataset})

In [16]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 100
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 43
    })
})

In [17]:
final_dataset['train'][855:865]

{'translation': [{'en': "You don't need to go in such a hurry.",
   'ur': 'تمھیں اتنی جلدی جانے کی ضرورت نہیں ہے۔'},
  {'en': 'Your mother will be back before long.',
   'ur': 'تمہاری امی بس آتی ہونگی۔'},
  {'en': 'Your mother will be back before long.',
   'ur': 'تمہاری ماں بس راستے میں ہو گی۔'},
  {'en': 'All of us were busy cleaning the room.',
   'ur': 'ہم سارے کمرے کی صفائی کرنے میں مصروف تھے۔'},
  {'en': 'All you have to do is sweep the floor.',
   'ur': 'تمہیں صرف زمین کی صفائی کرنی ہے۔'},
  {'en': 'Different people have different ideas.',
   'ur': 'مختلف لوگوں کے مختلف خیالات ہیں۔'},
  {'en': 'English is studied all over the world.',
   'ur': 'انگریزی پوری دنیا میں پڑھای جاتی ہے۔'},
  {'en': 'Everyone was listening very carefully.',
   'ur': 'سارے بڑے غور سے سن رہے تھے۔'},
  {'en': 'He does not take care of his children.',
   'ur': 'وہ اپنے بچوں کا خیال نہیں کرتا۔'},
  {'en': 'I have a lot of passwords to remember.',
   'ur': 'مجھے کافی زیادہ پاسورڈس یاد دکھنے پڑتے ہے۔'}]}

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [18]:
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):

        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(final_dataset["train"])

,translation
0,"{'en': 'All of us were busy cleaning the room.', 'ur': 'ہم سارے کمرے کی صفائی کرنے میں مصروف تھے۔'}"
1,"{'en': 'She's a frequent visitor to this country.', 'ur': 'وہ اس ملک میں آتی جاتی رہتی ہے۔'}"
2,"{'en': 'Tom is only a beginner.', 'ur': 'ٹام نے ابھی ابتدا کی ہے۔'}"
3,"{'en': 'Paris is the capital of France.', 'ur': 'پیرس فرانس کا دارلخلافہ ہے۔'}"
4,"{'en': 'You need to study more.', 'ur': 'تمھیں زیادہ پڑھنا چاہئیے۔'}"


We will use **metric** to evaluation (to compare our model to the benchmark). This can be easily done with the functions **load_metric**.

In [19]:
pip install sacrebleu

In [20]:
metric = load_metric("sacrebleu")

<ipython-input-20-aede95515358>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [21]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

You can call its **compute method** with your **predictions** and **labels**, which need to be **list of decoded strings** (list of list for the labels):

In [22]:
fake_preds = ["سنو ذرا", "hello there"]
fake_labels = [["سنو ذرا"], ["hello there"]]
metric.compute(predictions=fake_preds, references=fake_labels,)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing (Tokenization) the dataset

Before we can feed those texts to our model, we need to preprocess them. This is done by a **Transformers Tokenizer** which will (as the name indicates) **tokenize the inputs** (including **converting the tokens to their corresponding IDs** in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we **instantiate our tokenizer with the AutoTokenizer.from_pretrained method**, which will ensure:

we get a **tokenizer** that **corresponds** to the **model architecture** we want to use, we download the vocabulary used when pretraining this specific checkpoint. That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

If you downloaded the model manually, you can provide model present directory instead of model_checkpoint.

In [23]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [24]:
tokenizer.save_pretrained(my_dir+"save_tokenizer")

('/content/save_tokenizer/tokenizer_config.json',
 '/content/save_tokenizer/special_tokens_map.json',
 '/content/save_tokenizer/vocab.json',
 '/content/save_tokenizer/source.spm',
 '/content/save_tokenizer/target.spm',
 '/content/save_tokenizer/added_tokens.json')

In [25]:
tokenizer = AutoTokenizer.from_pretrained('./save_tokenizer/')

You can directly call this tokenizer on one sentence or a pair of sentences:

In [26]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[56936, 9004, 2, 19741, 2889, 73, 4695, 73, 66, 7061, 15423, 67, 0], [4117, 15061, 66, 22, 73, 55163, 3565, 73, 66, 7061, 15423, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the **targets for our model**, we need to tokenize them inside the **as_target_tokenizer** context manager. This will make sure the **tokenizer uses the special tokens** corresponding to the targets:

In [27]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[20475, 2, 140, 155, 7166, 67, 0], [416, 22, 474, 7166, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [28]:
print("Or")

tgt_text = ["Hello, this one sentence!", "This is another sentence."],
print(tokenizer(text_target = tgt_text))

Or
{'input_ids': [[20475, 2, 140, 155, 7166, 67, 416, 22, 474, 7166, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


We can then write the **function** that will **preprocess our samples**. We just feed them to the **tokenizer** with the **argument truncation=True**. This will ensure that an input **longer** that what the model selected can handle will be **truncated to the maximum length accepted by the model**. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [29]:
prefix = ""

max_input_length = 128
max_target_length = 128

source_lang = "ur"
target_lang = "en"

def preprocess_function(examples):
    """
    Getting Input and target
    """
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [30]:
preprocess_function(final_dataset['train'][:2])

{'input_ids': [[31585, 0], [349, 412, 0]], 'attention_mask': [[1, 1], [1, 1, 1]], 'labels': [[16918, 5, 0], [5569, 67, 0]]}

To **apply** this function on **all** the pairs of sentences in our dataset, we just use the **map** method of our **dataset** object we created earlier. This will **apply the function on all** the elements of all the splits in dataset, so our **training, validation and testing data** will be **preprocessed** in one single command

In [31]:
tokenized_datasets = final_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

## Fine-tuning the model
Now that our data is ready, we can **download the pretrained** model and fine-tune it. Since our task is of the **sequence-to-sequence** kind, we use the **AutoModelForSeq2SeqLM** class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [32]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Saving the Model

In [33]:
model.save_pretrained(my_dir+"save_model")

## Loading Model from Save Directory

In [34]:
model = AutoModelForSeq2SeqLM.from_pretrained('save_model')

To **instantiate** a **Seq2SeqTrainer**, we will need to define **three** more things. The most important is the **Seq2SeqTrainingArguments**, which is a class that contains all the **attributes to customize the training**. It requires one **folder name**, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [35]:
batch_size = 16

model_name = model_checkpoint.split("/")[-1]
print("Model name:",model_name)

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True
)

Model name: opus-mt-ur-en


In [36]:
args

Seq2SeqTrainingArguments(output_dir='opus-mt-ur-en-finetuned-ur-to-en', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=20, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='opus-mt-ur-en-finetuned-ur-to-en/runs/Dec23_17-47-29_aa60a29d35e0', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_st

Here we **set** the **evaluation** to be done at the **end of each epoch**, tweak the learning rate, use the **batch_size** defined at the top of the cell and customize the weight decay. Since the **Seq2SeqTrainer** will **save** the **model** regularly and our dataset is quite large, we tell it to make** three saves maximum**. Lastly, we use the **predict_with_generate** option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Model will **save** under **{model_name}-finetuned-{source_lang}-to-{target_lang}** directory

Then, we need a special kind of **data collator**, which will not only **pad** the **inputs** to the **maximum length** in the batch, but also the **labels**:

In [37]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our **Seq2SeqTrainer** is **how to compute the metrics** from the **predictions**. We need to define a function for this, which will just use the **metric** we loaded earlier, and we have to do a bit of **pre-processing to decode the predictions into texts**

In [38]:
import numpy as np

def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions = decoded_preds, references = decoded_labels)

    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}

    return result

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [39]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

We can now finetune our model by just calling the train method:

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.844176,20.894200,11.460000
2,No log,1.856031,20.552600,11.600000
3,No log,1.875082,19.975500,11.530000
4,No log,1.920830,21.512500,11.540000
5,No log,1.946952,21.512700,11.620000
6,No log,1.984723,20.375600,11.440000
7,No log,2.024149,19.510700,11.390000
8,0.861800,2.061152,21.012000,11.650000
9,0.861800,2.093473,20.179700,11.590000
10,0.861800,2.128457,20.273600,11.380000


TrainOutput(global_step=1260, training_loss=0.46416334727453806, metrics={'train_runtime': 4856.2511, 'train_samples_per_second': 4.118, 'train_steps_per_second': 0.259, 'total_flos': 70406903955456.0, 'train_loss': 0.46416334727453806, 'epoch': 20.0})

In [41]:
trainer.save_model()

In [42]:
import os
for dirname, _, filenames in os.walk('opus-mt-ur-en-finetuned-ur-to-en'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

opus-mt-ur-en-finetuned-ur-to-en/training_args.bin
opus-mt-ur-en-finetuned-ur-to-en/generation_config.json
opus-mt-ur-en-finetuned-ur-to-en/special_tokens_map.json
opus-mt-ur-en-finetuned-ur-to-en/model.safetensors
opus-mt-ur-en-finetuned-ur-to-en/source.spm
opus-mt-ur-en-finetuned-ur-to-en/config.json
opus-mt-ur-en-finetuned-ur-to-en/tokenizer_config.json
opus-mt-ur-en-finetuned-ur-to-en/target.spm
opus-mt-ur-en-finetuned-ur-to-en/vocab.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/rng_state.pth
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/training_args.bin
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/generation_config.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/special_tokens_map.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/model.safetensors
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/source.spm
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/scheduler.pt
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-500/optimizer.pt
opus-mt-ur-en-finetuned-ur-to-en/ch

Our fine tuned model already saved under opus-mt-ur-en-finetuned-ur-to-en/

Load the model and translate some text from english to romanian

our fine tune model is doing far better than pre-trained model and close to google translator

In [28]:
from transformers import MarianMTModel, MarianTokenizer

# Model name for Urdu to English translation
model_name = 'opus-mt-ur-en-finetuned-ur-to-en'

# Load pre-trained model and tokenizer
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Get input from the user
user_input = input("Enter the sentence to translate from Urdu to English: ")

# Translate the user input
translated = model.generate(**tokenizer([user_input], return_tensors="pt", padding=True))

# Decode the translated output
result = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Print the result
print("Original Urdu sentence:", user_input)
print("Translated English sentence:", result[0])



Enter the sentence to translate from Urdu to English: آپ کيا کررہے ہو؟
Original Urdu sentence: آپ کيا کررہے ہو؟
Translated English sentence: What are you doing?


In [47]:
pip install googletrans==4.0.0-rc1


In [49]:
from flask import Flask, render_template, request
from googletrans import Translator
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

translator = Translator()

# Define Flask route
@app.route('/', methods=['GET', 'POST'])
def index():
    translated_text = None

    if request.method == 'POST':
        try:
            input_text = request.form['input_text']
            if input_text:
                # Translate the input text from Urdu to English
                translated_text = translator.translate(input_text, src='ur', dest='en').text
            else:
                translated_text = "Type a sentence to translate"
        except Exception as e:
            return render_template('error.html', error_message=str(e))

    return render_template('index.html', translated_text=translated_text)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d4d7-34-150-170-188.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Dec/2023 22:30:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2023 22:30:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2023 22:30:59] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2023 22:31:47] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2023 22:32:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2023 22:32:35] "GET /favicon.ico HTTP/1.1" 404 -
